In [1]:
import os
import sys

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
# os.environ["NUMBA_DEBUG_FRONTEND"] = "1"
# os.environ["NUMBA_DEBUG"] = "1"
# os.environ["NUMBA_DEBUGINFO"] = "1"
# os.environ["NUMBA_SHOW_HELP"] = "1"
# os.environ["NUMBA_FULL_TRACEBACKS"] = "1"
# os.environ["NUMBA_DEVELOPER_MODE"] = "1"

In [2]:
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_base import backtest_df_only
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import nb_dos_cart_product
from nb_quantfreedom.strategies.nb_strategy import nb_BacktestInd, nb_Strategy


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=200,
)

It took 00 mins and 00 seconds to download 200 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([3]),
    max_trades=np.array([5]),
    num_candles=np.array([60]),
    risk_account_pct_size=np.array([3, 5]),
    risk_reward=np.array([3]),
    sl_based_on_add_pct=np.array([.01]),
    sl_based_on_lookback=np.array([10]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1.20]),
    trail_sl_when_pct=np.array([.001]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = nb_dos_cart_product(
    dos_arrays=dos_arrays,
)


In [5]:
backtest_df_only(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_cart_arrays=dos_cart_arrays,
    exchange_settings=mufex_main.exchange_settings,
    # logger_settings=None,
    logger_settings='p',
    # logger_settings=LoggerSettings(log_debug=True, create_trades_logger=False),
    starting_equity=1000.0,
    static_os=static_os,
    strategy=nb_Strategy(),
    ind_creator=nb_BacktestInd(),
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 2
Total combinations of settings to test: 8
Total candles: 200
Total candles to test: 1600
nb_strategy.py - nb_Strategy - nb_get_current_ind_settings() - Created indicator settings
nb_simulate.py - No Class - nb_run_backtest() - Indicator settings index=0
nb_simulate.py - No Class - nb_run_backtest() - Indicator settings
RSI Period= 14 RSI is below= 
nb_simulate.py - No Class - nb_run_backtest() - Dynamic Order settings index=0
nb_simulate.py - No Class - nb_run_backtest() - Starting Bar=59
nb_simulate.py - No Class - nb_run_backtest() - account state order results pnl array all set to default



nb_simulate.py - No Class - nb_run_backtest() - ind_idx=0 dos_idx=0 bar_idx=59 timestamp=1697810400000
nb_simulate.py - No Class - nb_run_backtest() - Not in a pos so not checking SL Liq or TP
nb_simulate.py - No Class - nb_run_backtest() - strategy e

,ind_set_idx,dos_index,total_trades,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,5.0,8.42,40.0,0.0,60.627,84.154,1084.154
1,1,0,2.0,5.69,50.0,1.0,15.232,56.944,1056.944
2,2,0,2.0,5.69,50.0,1.0,17.756,56.929,1056.929
3,3,0,2.0,5.69,50.0,1.0,17.756,56.929,1056.929


In [6]:
type(round(10.49859))

int